<a href="https://colab.research.google.com/github/youssafB/Time_series_Transformer/blob/main/gluontranformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

In [2]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [3]:
!pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


In [4]:
!pip install -q gluonts ujson

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.7 MB/s eta 0:00:00


In [18]:
from typing import Iterable, Any
from gluonts.transform import Transformation
from gluonts.dataset.field_names import FieldName
from gluonts.time_feature import time_features_from_frequency_str
from gluonts.transform import RemoveFields, AsNumpyArray,AddObservedValuesIndicator,AddTimeFeatures,VstackFeatures,RenameFields


# Step 0 :GluonTS `FieldName` Arguments:

In [20]:
from gluonts.dataset.field_names import FieldName
for fieled in dir(FieldName):
  if not fieled.startswith("__") :
      print(fieled)

FEAT_AGE
FEAT_CONST
FEAT_DYNAMIC
FEAT_DYNAMIC_CAT
FEAT_DYNAMIC_REAL
FEAT_DYNAMIC_REAL_LEGACY
FEAT_STATIC_CAT
FEAT_STATIC_REAL
FEAT_TIME
FORECAST_START
INFO
IS_PAD
ITEM_ID
OBSERVED_VALUES
PAST_FEAT_DYNAMIC
PAST_FEAT_DYNAMIC_CAT
PAST_FEAT_DYNAMIC_REAL
START
TARGET
TARGET_DIM_INDICATOR



| Field Name                 | Required | Description |
|----------------------------|----------|-------------|
| `START`                   | ✅ Yes  | The start time of the time series. |
| `TARGET`                  | ✅ Yes  | The target time series values (what you want to predict). |
| `FORECAST_START`          | ❌ No   | The start time of the forecast horizon. |
| `FEAT_AGE`                | ❌ No   | Age feature, often used for transformations. |
| `FEAT_CONST`              | ❌ No   | Constant feature values across all time steps. |
| `FEAT_DYNAMIC`            | ❌ No   | General dynamic feature placeholder. |
| `FEAT_DYNAMIC_CAT`        | ❌ No   | Dynamic categorical features (change over time). |
| `FEAT_DYNAMIC_REAL`       | ❌ No   | Dynamic real-valued features (change over time). |
| `FEAT_DYNAMIC_REAL_LEGACY`| ❌ No   | Legacy version of `FEAT_DYNAMIC_REAL`. |
| `FEAT_STATIC_CAT`         | ❌ No   | Categorical static features (do not change over time). |
| `FEAT_STATIC_REAL`        | ❌ No   | Real-valued static features. |
| `FEAT_TIME`               | ❌ No   | Time-based features (e.g., hour, day of the week). |
| `INFO`                    | ❌ No   | Metadata or additional information. |
| `IS_PAD`                  | ❌ No   | Marks padded values in a batch (used for batching). |
| `ITEM_ID`                 | ❌ No   | Unique identifier for each time series. |
| `OBSERVED_VALUES`         | ❌ No   | Indicates missing values in `TARGET` (1 = observed, 0 = missing). |
| `PAST_FEAT_DYNAMIC`       | ❌ No   | Dynamic features from past values. |
| `PAST_FEAT_DYNAMIC_CAT`   | ❌ No   | Past dynamic categorical features. |
| `PAST_FEAT_DYNAMIC_REAL`  | ❌ No   | Past dynamic real-valued features. |


# Part 1: Building a Small Dataset in GluonTS Format and Configuration


In [ ]:
# Sample Data
sample_data = [
    {
        FieldName.TARGET: [[1, 2, np.nan], [4, 5, np.nan]],
        FieldName.START: pd.Period("2023-01", freq="M"),
        FieldName.FEAT_STATIC_REAL: [0.5],
        FieldName.FEAT_DYNAMIC_REAL: [1.0, 2.0, 3.0],
        FieldName.FEAT_STATIC_CAT: [10],
    },
    {
        FieldName.TARGET: [[2, 3, 4], [5, 6, 7]],
        FieldName.START: pd.Period("2024-01", freq="M"),
        FieldName.FEAT_STATIC_REAL: [0.7],
        FieldName.FEAT_DYNAMIC_REAL: [1.5, 2.5, 3.5],
        FieldName.FEAT_STATIC_CAT: [20],
    }
]

# Example config to remove static categorical and dynamic real features
config = {
    "num_static_real_features": 0,  # Remove
    "num_dynamic_real_features": 1,  # keep
    "num_static_categorical_features": 0,  # Remove
}

In [ ]:
print(sample_data)

[{'target': [[1, 2, nan], [4, 5, nan]], 'start': Period('2023-01', 'M'), 'feat_static_real': [0.5], 'feat_dynamic_real': [1.0, 2.0, 3.0], 'feat_static_cat': [10]}, {'target': [[2, 3, 4], [5, 6, 7]], 'start': Period('2024-01', 'M'), 'feat_static_real': [0.7], 'feat_dynamic_real': [1.5, 2.5, 3.5], 'feat_static_cat': [20]}]


## 1. 🧹 Field Cleanup System


**🧹 Field Removal Logic**: Automatically detects unused data fields based on config.

In [ ]:
from gluonts.transform import RemoveFields
from typing import List, Dict, Any
from gluonts.dataset.field_names import FieldName

def determine_fields_to_remove(config: Dict[str, Any]) -> List[str]:
    """
    Determines which fields should be removed based on the config.

    Args:
        config (Dict[str, Any]): Configuration dictionary with keys that specify feature counts.

    Returns:
        List[str]: A list of field names to remove.
    """
    fields_to_remove = []
    if config.get("num_static_real_features", 0) == 0:
        fields_to_remove.append(FieldName.FEAT_STATIC_REAL)
    if config.get("num_dynamic_real_features", 0) == 0:
        fields_to_remove.append(FieldName.FEAT_DYNAMIC_REAL)
    if config.get("num_static_categorical_features", 0) == 0:
        fields_to_remove.append(FieldName.FEAT_STATIC_CAT)
    return fields_to_remove

In [ ]:
features_to_remove = determine_fields_to_remove(config)
print("Fields to remove:", features_to_remove )


Fields to remove: ['feat_static_real', 'feat_static_cat']


In [ ]:
features_to_remove_args = {
    "num_static_real_features": 0,  # Replace with actual value
    "num_dynamic_real_features": 0,  # Replace with actual value
    "num_static_categorical_features": 0  # Replace with actual value
}

##  2 ✂️ RemoveFieldsTask:  Data Trimming Tool

In [ ]:

from gluonts.transform import Transformation

class RemoveFieldsTask(RemoveFields):
    """
    Wrapper for the RemoveFields transformer that initializes with specified field names
    and applies the transformation when called.
    """
    def __init__(self, config: Dict[str, Any]):
        """
        Initializes the RemoveFieldsTask using a config dictionary.

        Args:
            config (Dict[str, Any]): Configuration for determining fields to remove.
        """
        field_names = determine_fields_to_remove(config)
        self.transformer = RemoveFields(field_names=field_names)

    def __call__(self, data, is_train: bool):
        """Apply the transformation and return the result."""
        return self.transformer(data, is_train=is_train)


In [ ]:
class RemoveFieldsTask(RemoveFields):
    """
    Wrapper for the RemoveFields transformer that initializes with specified field names
    and applies the transformation when called.
    """
    def __init__(self, config: Dict[str, Any]):
        """

        Args:
            config (Dict[str, Any]): Configuration for determining fields to remove.
        """
        # Determine the fields to remove based on the configuration
        field_names = determine_fields_to_remove(config)

        # Call the original RemoveFields class with the processed arguments
        super().__init__(field_names=field_names)

    def __call__(self, data, is_train: bool):
        """Apply the transformation and return the result."""
        # Simply call the base class method with the data and is_train arguments
        return super().__call__(data, is_train=is_train)

### 🔄 Transformation Process

In [ ]:

remove_fields_task = RemoveFieldsTask(config)

# Apply the transformation
transformed_data = list(remove_fields_task.apply(sample_data, is_train=True))


## ✅  Print the result
print("Transformed data:", transformed_data[1])

Transformed data: {'target': [[2, 3, 4], [5, 6, 7]], 'start': Period('2024-01', 'M'), 'feat_dynamic_real': [1.5, 2.5, 3.5]}


###🧼 Display Before & After

In [ ]:
# Display the input and output.
print("Input Data:")
for i, item in enumerate(sample_data, start=1):
    print(f"Item {i}: {item}")

print("\nTransformed Data (after removing specified fields):")
for i, item in enumerate(transformed_data, start=1):
    print(f"Item {i}: {item}")

Input Data:
Item 1: {'target': [[1, 2, nan], [4, 5, nan]], 'start': Period('2023-01', 'M'), 'feat_static_real': [0.5], 'feat_dynamic_real': [1.0, 2.0, 3.0], 'feat_static_cat': [10]}
Item 2: {'target': [[2, 3, 4], [5, 6, 7]], 'start': Period('2024-01', 'M'), 'feat_static_real': [0.7], 'feat_dynamic_real': [1.5, 2.5, 3.5], 'feat_static_cat': [20]}

Transformed Data (after removing specified fields):
Item 1: {'target': [[1, 2, nan], [4, 5, nan]], 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0]}
Item 2: {'target': [[2, 3, 4], [5, 6, 7]], 'start': Period('2024-01', 'M'), 'feat_dynamic_real': [1.5, 2.5, 3.5]}


In [ ]:
remove_fields_task = RemoveFieldsTask(config)

transformer_chain = Chain([remove_fields_task])

# Apply the transformation to each sample using the chain
transformed_data = list(transformer_chain(sample_data, is_train=True))

# Display the result
print("After RemoveFields Transformation:", transformed_data[0])

After RemoveFields Transformation: {'target': [[1, 2, nan], [4, 5, nan]], 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0]}


# Part 2

In [ ]:
from gluonts.transform import AsNumpyArray
from typing import Dict, Any



class AsNumpyArrayTask(AsNumpyArray):
    """
    Wrapper for AsNumpyArray transformation.
    Allows easy initialization via a config dictionary.
    """

    def __init__(self, config: Dict[str, Any]):
        """

        """
        # Extract the parameters from the configuration dictionary
        field = config.get("field")
        expected_ndim = config.get("expected_ndim", 1)
        dtype = config.get("dtype", float)

        # Ensure the 'field' is provided in the configuration
        if field is None:
            raise ValueError("Config must contain a 'field' key.")

        # Initialize the AsNumpyArray transformer with the parameters
        self.transformer = AsNumpyArray(field=field, expected_ndim=expected_ndim, dtype=dtype)

    def __call__(self, data, is_train: bool, **kwargs):
        """Apply the transformation and return the result."""
        return self.transformer(data, is_train=is_train, **kwargs)


In [ ]:
# Define the configuration
my_config = {"field": "target", "expected_ndim": 2, "dtype": float}

# Initialize the wrapper
as_numpy_array_task = AsNumpyArrayTask(my_config)

# Apply the transformation (make sure to pass is_train)
transformed_data = list(as_numpy_array_task(sample_data, is_train=True))

print("Transformed data:", transformed_data[1])


Transformed data: {'target': array([[2., 3., 4.],
       [5., 6., 7.]]), 'start': Period('2024-01', 'M'), 'feat_static_real': [0.7], 'feat_dynamic_real': [1.5, 2.5, 3.5], 'feat_static_cat': [20]}


In [ ]:
# Display the input and output.
print("Input Data:")
for i, item in enumerate(sample_data, start=1):
    print(f"Item {i}: {item}")

print("\nTransformed Data :")
for i, item in enumerate(transformed_data, start=1):
    print(f"Item {i}: {item}")

Input Data:
Item 1: {'target': [[1, 2, nan], [4, 5, nan]], 'start': Period('2023-01', 'M'), 'feat_static_real': [0.5], 'feat_dynamic_real': [1.0, 2.0, 3.0], 'feat_static_cat': [10]}
Item 2: {'target': [[2, 3, 4], [5, 6, 7]], 'start': Period('2024-01', 'M'), 'feat_static_real': [0.7], 'feat_dynamic_real': [1.5, 2.5, 3.5], 'feat_static_cat': [20]}

Transformed Data :
Item 1: {'target': array([[ 1.,  2., nan],
       [ 4.,  5., nan]]), 'start': Period('2023-01', 'M'), 'feat_static_real': [0.5], 'feat_dynamic_real': [1.0, 2.0, 3.0], 'feat_static_cat': [10]}
Item 2: {'target': array([[2., 3., 4.],
       [5., 6., 7.]]), 'start': Period('2024-01', 'M'), 'feat_static_real': [0.7], 'feat_dynamic_real': [1.5, 2.5, 3.5], 'feat_static_cat': [20]}


## in the chain

In [ ]:
remove_fields_task = RemoveFieldsTask(config)
as_numpy_array_task = AsNumpyArrayTask(my_config)

transformer_chain = Chain([remove_fields_task, as_numpy_array_task])

# Apply the transformation to each sample using the chain
transformed_data = list(transformer_chain.apply(sample_data, is_train=True))

# Display the result
print("After RemoveFields Transformation:", transformed_data[1])

After RemoveFields Transformation: {'target': array([[2., 3., 4.],
       [5., 6., 7.]]), 'start': Period('2024-01', 'M'), 'feat_dynamic_real': [1.5, 2.5, 3.5]}


In [ ]:
# Display the input and output.
print("Input Data:")
for i, item in enumerate(sample_data, start=1):
    print(f"Item {i}: {item}")

print("\nTransformed Data :")
for i, item in enumerate(transformed_data, start=1):
    print(f"Item {i}: {item}")

Input Data:
Item 1: {'target': [[1, 2, nan], [4, 5, nan]], 'start': Period('2023-01', 'M'), 'feat_static_real': [0.5], 'feat_dynamic_real': [1.0, 2.0, 3.0], 'feat_static_cat': [10]}
Item 2: {'target': [[2, 3, 4], [5, 6, 7]], 'start': Period('2024-01', 'M'), 'feat_static_real': [0.7], 'feat_dynamic_real': [1.5, 2.5, 3.5], 'feat_static_cat': [20]}

Transformed Data :
Item 1: {'target': array([[ 1.,  2., nan],
       [ 4.,  5., nan]]), 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0]}
Item 2: {'target': array([[2., 3., 4.],
       [5., 6., 7.]]), 'start': Period('2024-01', 'M'), 'feat_dynamic_real': [1.5, 2.5, 3.5]}


# Part 3

In [ ]:


from gluonts.transform import AddObservedValuesIndicator
from gluonts.dataset.common import FieldName
from typing import Any

class HandleNaNsAndAddMask(AddObservedValuesIndicator):
    """
    Wrapper for AddObservedValuesIndicator that initializes with fixed target_field and output_field.
    Allows handling NaNs and adding a mask for observed values.
    """

    def __init__(self, target_field=FieldName.TARGET, output_field=FieldName.OBSERVED_VALUES):
        """
        Initializes HandleNaNsAndAddMask with fixed target_field and output_field.

        Args:
            target_field (str): The field name to apply AddObservedValuesIndicator on.
            output_field (str): The output field name to store the mask.
        """
        # Call the parent class constructor with the appropriate parameters
        super().__init__(target_field=target_field, output_field=output_field)

    def __call__(self, data: Any, is_train: bool, **kwargs) -> Any:
        """Apply the transformation and return the result."""
        # Simply call the parent class method
        return super().__call__(data, is_train=is_train, **kwargs)



In [ ]:
# Initialize the wrapper with default field names
handle_nans_task = HandleNaNsAndAddMask()

# Apply the transformation (make sure to pass is_train)
transformed_data = list(handle_nans_task(transformed_data, is_train=True))

print("Transformed data:", transformed_data[0])


Transformed data: {'target': array([[1., 2., 0.],
       [4., 5., 0.]]), 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0], 'observed_values': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32)}


### chain

In [ ]:
remove_fields_task = RemoveFieldsTask(config)
as_numpy_array_task = AsNumpyArrayTask(my_config)
handle_nans_task = HandleNaNsAndAddMask() # it has defult arguemtns

transformer_chain = Chain([remove_fields_task, as_numpy_array_task, handle_nans_task])

# Apply the transformation to each sample using the chain
transformed_data = list(transformer_chain.apply(sample_data, is_train=True))

# Display the result
print("After RemoveFields Transformation:", transformed_data[0])

After RemoveFields Transformation: {'target': array([[1., 2., 0.],
       [4., 5., 0.]]), 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0], 'observed_values': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32)}


In [ ]:
# Display the input and output.
print("Input Data:")
for i, item in enumerate(sample_data, start=1):
    print(f"Item {i}: {item}")

print("\nTransformed Data :")
for i, item in enumerate(transformed_data, start=1):
    print(f"Item {i}: {item}")

Input Data:
Item 1: {'target': [[1, 2, nan], [4, 5, nan]], 'start': Period('2023-01', 'M'), 'feat_static_real': [0.5], 'feat_dynamic_real': [1.0, 2.0, 3.0], 'feat_static_cat': [10]}
Item 2: {'target': [[2, 3, 4], [5, 6, 7]], 'start': Period('2024-01', 'M'), 'feat_static_real': [0.7], 'feat_dynamic_real': [1.5, 2.5, 3.5], 'feat_static_cat': [20]}

Transformed Data :
Item 1: {'target': array([[1., 2., 0.],
       [4., 5., 0.]]), 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0], 'observed_values': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32)}
Item 2: {'target': array([[2., 3., 4.],
       [5., 6., 7.]]), 'start': Period('2024-01', 'M'), 'feat_dynamic_real': [1.5, 2.5, 3.5], 'observed_values': array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)}


# 4

In [ ]:
from gluonts.transform import AddTimeFeatures
from gluonts.dataset.common import FieldName
from gluonts.time_feature import time_features_from_frequency_str
from typing import Any

class AddTemporalFeatures(AddTimeFeatures):
    """
    Wrapper for AddTimeFeatures transformation.
    Adds temporal features such as the month of the year based on the frequency of the dataset.
    Only `freq` and `pred_length` are needed as arguments.
    """

    def __init__(self, freq: str, pred_length: int, **kwargs):
        """
        Initializes AddTemporalFeatures with fixed arguments for time features.

        Args:
            freq (str): The frequency of the data (e.g., 'D' for daily, 'H' for hourly).
            pred_length (int): The prediction length for the model.
        """
        # Determine the time features from the frequency string
        time_features = time_features_from_frequency_str(freq)

        # Call the parent class constructor with fixed field names and time features
        super().__init__(
            start_field=FieldName.START,
            target_field=FieldName.TARGET,
            output_field=FieldName.FEAT_TIME,
            time_features=time_features,
            pred_length=pred_length,  # Prediction length passed here
            **kwargs
        )

    def __call__(self, data: Any, is_train: bool, **kwargs) -> Any:
        """Apply the transformation and return the result."""
        # Simply call the parent class's __call__ method
        return super().__call__(data, is_train=is_train, **kwargs)


In [ ]:
# Initialize the wrapper with required fields and arguments
add_temporal_features_task = AddTemporalFeatures(freq='D', pred_length=24)

# Apply the transformation (make sure to pass is_train)
transformed_data = list(add_temporal_features_task(transformed_data, is_train=True))

print("Transformed data:", transformed_data[1])


Transformed data: {'target': array([[2., 3., 4.],
       [5., 6., 7.]]), 'start': Period('2024-01', 'M'), 'feat_dynamic_real': [1.5, 2.5, 3.5], 'observed_values': array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32), 'time_feat': array([[-0.16666667,  0.        ,  0.5       ],
       [ 0.5       ,  0.43333334,  0.5       ],
       [-0.4178082 , -0.33835617, -0.25342464]], dtype=float32)}


In [ ]:
remove_fields_task = RemoveFieldsTask(config)
as_numpy_array_task = AsNumpyArrayTask(my_config)
handle_nans_task = HandleNaNsAndAddMask() # it has defult arguemtns
add_temporal_features_task = AddTemporalFeatures(freq='D', pred_length=24)

transformer_chain = Chain([remove_fields_task, as_numpy_array_task, handle_nans_task, add_temporal_features_task])

# Apply the transformation to each sample using the chain
transformed_data = list(transformer_chain.apply(sample_data, is_train=True))

# Display the result
print("After RemoveFields Transformation:", transformed_data[0])

After RemoveFields Transformation: {'target': array([[1., 2., 0.],
       [4., 5., 0.]]), 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0], 'observed_values': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32), 'time_feat': array([[-0.33333334, -0.33333334,  0.16666667],
       [ 0.5       ,  0.4       ,  0.5       ],
       [-0.4178082 , -0.3410959 , -0.25616437]], dtype=float32)}


In [ ]:
# Display the input and output.
print("Input Data:")
for i, item in enumerate(sample_data, start=1):
    print(f"Item {i}: {item}")

print("\nTransformed Data :")
for i, item in enumerate(transformed_data, start=1):
    print(f"Item {i}: {item}")

Input Data:
Item 1: {'target': [[1, 2, nan], [4, 5, nan]], 'start': Period('2023-01', 'M'), 'feat_static_real': [0.5], 'feat_dynamic_real': [1.0, 2.0, 3.0], 'feat_static_cat': [10]}
Item 2: {'target': [[2, 3, 4], [5, 6, 7]], 'start': Period('2024-01', 'M'), 'feat_static_real': [0.7], 'feat_dynamic_real': [1.5, 2.5, 3.5], 'feat_static_cat': [20]}

Transformed Data :
Item 1: {'target': array([[1., 2., 0.],
       [4., 5., 0.]]), 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0], 'observed_values': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32), 'time_feat': array([[-0.33333334, -0.33333334,  0.16666667],
       [ 0.5       ,  0.4       ,  0.5       ],
       [-0.4178082 , -0.3410959 , -0.25616437]], dtype=float32)}
Item 2: {'target': array([[2., 3., 4.],
       [5., 6., 7.]]), 'start': Period('2024-01', 'M'), 'feat_dynamic_real': [1.5, 2.5, 3.5], 'observed_values': array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32), 'time_feat': array([[-0.16666667, 

#5

In [ ]:
from gluonts.transform import AddAgeFeature
from gluonts.dataset.common import FieldName
from typing import Iterable, Any

class AddAgeFeatures(AddAgeFeature):
    """
    Wrapper for AddAgeFeature transformation.
    Adds an age feature to the time series, indicating the position of each time step.
    Only `pred_length` and `log_scale` are needed as arguments.
    """

    def __init__(self, pred_length: int, log_scale: bool = True, **kwargs):
        """
        Initializes AddAgeFeatures with fixed arguments for age feature.

        Args:
            pred_length (int): The prediction length for the model.
            log_scale (bool): Whether to apply log scaling to the age feature.
        """
        # Initialize the parent class with required parameters
        super().__init__(
            target_field=FieldName.TARGET,
            output_field=FieldName.FEAT_AGE,
            pred_length=pred_length,
            log_scale=log_scale,
            **kwargs
        )

    def __call__(self, data: Iterable[Any], is_train: bool, **kwargs) -> Iterable[Any]:
        """Apply the transformation and return the result."""
        return super().__call__(data, is_train=is_train, **kwargs)


In [ ]:
# Initialize the wrapper with required arguments
add_age_features_task = AddAgeFeatures(pred_length=24, log_scale=True)

# Apply the transformation (make sure to pass is_train)
transformed_data = list(add_age_features_task(transformed_data, is_train=True))

print("Transformed data:", transformed_data[0])


Transformed data: {'target': array([[1., 2., 0.],
       [4., 5., 0.]]), 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0], 'observed_values': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32), 'time_feat': array([[-0.33333334, -0.33333334,  0.16666667],
       [ 0.5       ,  0.4       ,  0.5       ],
       [-0.4178082 , -0.3410959 , -0.25616437]], dtype=float32), 'feat_dynamic_age': array([[0.30103   , 0.47712123, 0.60206   ]], dtype=float32)}


In [ ]:
remove_fields_task = RemoveFieldsTask(config)
as_numpy_array_task = AsNumpyArrayTask(my_config)
handle_nans_task = HandleNaNsAndAddMask() # it has defult arguemtns
add_temporal_features_task = AddTemporalFeatures(freq='D', pred_length=24)
add_age_features_task = AddAgeFeatures(pred_length=24, log_scale=True)

transformer_chain = Chain([remove_fields_task, as_numpy_array_task, handle_nans_task, add_temporal_features_task, add_age_features_task ])

# Apply the transformation to each sample using the chain
transformed_data = list(transformer_chain.apply(sample_data, is_train=True))

# Display the result
print("After RemoveFields Transformation:", transformed_data[0])

After RemoveFields Transformation: {'target': array([[1., 2., 0.],
       [4., 5., 0.]]), 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0], 'observed_values': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32), 'time_feat': array([[-0.33333334, -0.33333334,  0.16666667],
       [ 0.5       ,  0.4       ,  0.5       ],
       [-0.4178082 , -0.3410959 , -0.25616437]], dtype=float32), 'feat_dynamic_age': array([[0.30103   , 0.47712123, 0.60206   ]], dtype=float32)}


In [ ]:
# Display the input and output.
print("Input Data:")
for i, item in enumerate(sample_data, start=1):
    print(f"Item {i}: {item}")

print("\nTransformed Data :")
for i, item in enumerate(transformed_data, start=1):
    print(f"Item {i}: {item}")

Input Data:
Item 1: {'target': [[1, 2, nan], [4, 5, nan]], 'start': Period('2023-01', 'M'), 'feat_static_real': [0.5], 'feat_dynamic_real': [1.0, 2.0, 3.0], 'feat_static_cat': [10]}
Item 2: {'target': [[2, 3, 4], [5, 6, 7]], 'start': Period('2024-01', 'M'), 'feat_static_real': [0.7], 'feat_dynamic_real': [1.5, 2.5, 3.5], 'feat_static_cat': [20]}

Transformed Data :
Item 1: {'target': array([[1., 2., 0.],
       [4., 5., 0.]]), 'start': Period('2023-01', 'M'), 'feat_dynamic_real': [1.0, 2.0, 3.0], 'observed_values': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32), 'time_feat': array([[-0.33333334, -0.33333334,  0.16666667],
       [ 0.5       ,  0.4       ,  0.5       ],
       [-0.4178082 , -0.3410959 , -0.25616437]], dtype=float32), 'feat_dynamic_age': array([[0.30103   , 0.47712123, 0.60206   ]], dtype=float32)}
Item 2: {'target': array([[2., 3., 4.],
       [5., 6., 7.]]), 'start': Period('2024-01', 'M'), 'feat_dynamic_real': [1.5, 2.5, 3.5], 'observed_values': array([[1.,

#6

In [ ]:
from gluonts.transform import VstackFeatures
from gluonts.dataset.common import FieldName
from typing import Iterable, Any
import pandas as pd  # Ensure pandas is imported for handling Periods

class VstackTemporalFeatures(VstackFeatures):
    """
    Wrapper for VstackFeatures transformation.
    Vertically stacks temporal features (e.g., FEAT_TIME, FEAT_AGE) into the output field `FEAT_TIME`.
    """

    def __init__(self, num_dynamic_real_features: int, **kwargs):
        """
        Initializes VstackTemporalFeatures with input fields and dynamic features.

        Args:
            num_dynamic_real_features (int): The number of dynamic real features.
        """
        # Determine the input fields, including dynamic features if present
        input_fields = [FieldName.FEAT_TIME, FieldName.FEAT_AGE]

        if num_dynamic_real_features > 0:
            input_fields.append(FieldName.FEAT_DYNAMIC_REAL)

        # Initialize the VstackFeatures transformer with input fields
        super().__init__(
            output_field=FieldName.FEAT_TIME,
            input_fields=input_fields,
            **kwargs
        )

    def __call__(self, data: Iterable[Any], is_train: bool, **kwargs) -> Iterable[Any]:
        """Apply the transformation and return the result."""
        return super().__call__(data,is_train=is_train, **kwargs)



In [ ]:
# Initialize the wrapper with required arguments
vstack_features_task = VstackTemporalFeatures(num_dynamic_real_features=3)

# Apply the transformation (make sure to pass is_train)
transformed_data = list(vstack_features_task(transformed_data, is_train=True))

print("Transformed data:", transformed_data[0])


KeyError: 'feat_dynamic_age'

In [ ]:
remove_fields_task = RemoveFieldsTask(config)
as_numpy_array_task = AsNumpyArrayTask(my_config)
handle_nans_task = HandleNaNsAndAddMask() # it has defult arguemtns
add_temporal_features_task = AddTemporalFeatures(freq='D', pred_length=24)
add_age_features_task = AddAgeFeatures(pred_length=24, log_scale=True)
vstack_features_task = VstackTemporalFeatures(num_dynamic_real_features=3)

transformer_chain = Chain([remove_fields_task, as_numpy_array_task, handle_nans_task, add_temporal_features_task, add_age_features_task, vstack_features_task ])

# Apply the transformation to each sample using the chain
transformed_data = list(transformer_chain.apply(sample_data, is_train=True))

# Display the result
print("After RemoveFields Transformation:", transformed_data[0])

After RemoveFields Transformation: {'target': array([[1., 2., 0.],
       [4., 5., 0.]]), 'start': Period('2023-01', 'M'), 'observed_values': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32), 'time_feat': array([[-0.33333334, -0.33333334,  0.16666667],
       [ 0.5       ,  0.40000001,  0.5       ],
       [-0.4178082 , -0.34109589, -0.25616437],
       [ 0.30103001,  0.47712123,  0.60206002],
       [ 1.        ,  2.        ,  3.        ]])}


#7

In [ ]:
from gluonts.transform import RenameFields
from gluonts.dataset.common import FieldName
from typing import Iterable, Any

class RenameTemporalFeatures(RenameFields):
    """
    Wrapper for RenameFields transformation to rename fields to match HuggingFace names.
    The field names are predefined in a dictionary for mapping.
    """

    def __init__(self, **kwargs):
        """
        Initializes RenameTemporalFeatures with a predefined field name mapping.

        Args:
            **kwargs: Any additional arguments for customization if needed.
        """
        # Define the field name mapping
        mapping = {
            FieldName.FEAT_STATIC_CAT: "static_categorical_features",
            FieldName.FEAT_STATIC_REAL: "static_real_features",
            FieldName.FEAT_TIME: "time_features",
            FieldName.TARGET: "values",
            FieldName.OBSERVED_VALUES: "observed_mask",
        }

        # Initialize the RenameFields transformer with the mapping
        super().__init__(mapping=mapping, **kwargs)

    def __call__(self, data: Iterable[Any], is_train: bool, **kwargs) -> Iterable[Any]:
        """Apply the transformation and return the result."""
        return super().__call__(data, is_train=is_train, **kwargs)


In [ ]:
# Initialize the wrapper with necessary arguments
rename_temporal_features_task = RenameTemporalFeatures()

# Apply the transformation (make sure to pass is_train)
transformed_data = list(rename_temporal_features_task(transformed_data, is_train=True))

print("Transformed data:", transformed_data[0])


Transformed data: {'start': Period('2023-01', 'M'), 'time_features': array([[-0.33333334, -0.33333334,  0.16666667],
       [ 0.5       ,  0.40000001,  0.5       ],
       [-0.4178082 , -0.34109589, -0.25616437],
       [ 0.30103001,  0.47712123,  0.60206002],
       [ 1.        ,  2.        ,  3.        ]]), 'values': array([[1., 2., 0.],
       [4., 5., 0.]]), 'observed_mask': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32)}


In [ ]:
remove_fields_task = RemoveFieldsTask(config)
as_numpy_array_task = AsNumpyArrayTask(my_config)
handle_nans_task = HandleNaNsAndAddMask() # it has defult arguemtns
add_temporal_features_task = AddTemporalFeatures(freq='D', pred_length=24)
add_age_features_task = AddAgeFeatures(pred_length=24, log_scale=True)
vstack_features_task = VstackTemporalFeatures(num_dynamic_real_features=3)
rename_temporal_features_task = RenameTemporalFeatures()

transformer_chain = Chain([remove_fields_task, as_numpy_array_task, handle_nans_task, add_temporal_features_task,
                           add_age_features_task, vstack_features_task,rename_temporal_features_task  ])

# Apply the transformation to each sample using the chain
transformed_data = list(transformer_chain.apply(sample_data, is_train=True))

# Display the result
print("After RemoveFields Transformation:", transformed_data[0])

After RemoveFields Transformation: {'start': Period('2023-01', 'M'), 'time_features': array([[-0.33333334, -0.33333334,  0.16666667],
       [ 0.5       ,  0.40000001,  0.5       ],
       [-0.4178082 , -0.34109589, -0.25616437],
       [ 0.30103001,  0.47712123,  0.60206002],
       [ 1.        ,  2.        ,  3.        ]]), 'values': array([[1., 2., 0.],
       [4., 5., 0.]]), 'observed_mask': array([[1., 1., 0.],
       [1., 1., 0.]], dtype=float32)}
